<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/OSI-SAF_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_?_link_to_prev_notebook.ipynb" target="_blank"><< PREV NOTEBOOK</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_?_link_to_next_notebook.ipynb" target="_blank">NEXT NOTEBOOK >></a>

<font color="#138D75">**EUMETSAT OSI SAF Training Service**</font> <br>
**Copyright:** 2022 EUMETSAT <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h3>Learn OSI SAF sea surface temperature: Introductory</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
This notebook has the following prerequisites:
- **COMPLETE WITH ANY PRE-REQUISITES (e.g. account access), IF ANY**

There are no prerequisite notebooks for this module.
- **COMPLETE WITH ANY PRE-REQUISITE NOTEBOOKS, IF ANY**
    
</div>
<hr>

# 1.1 Access Metop SST Metagranules - IFREMER Opensearch
### Data used 

| Product Description | Product Navigator | OSI SAF website link | OSI SAF Identifier |
|:--------------------:|:-------------:|:-----------------:|:-----------------:|
| Full resolution Metop-B Sea Surface Temperature metagranules | EO:EUM:DAT:METOP:MGR-SST | <a href="https://osi-saf.eumetsat.int/products/osi-204-b" target="_blank">link</a>| OSI-204-b | 
| Full resolution Metop-C Sea Surface Temperature metagranules | EO:EUM:DAT:METOP:MGR-SST | <a href="https://osi-saf.eumetsat.int/products/osi-204-c" target="_blank">link</a>| OSI-204-c | 


### Learning outcomes

At the end of this notebook you will know;
* What the GHRSST format is
* What components are inside a GHRSST format file from OSI SAF

### Outline 

All OSI SAF SST are delivered in netCDF and follow the "**GHRSST format**". The Group for High Resolution Sea Surface Temperature (GHRSST) is an open international science group that promotes the application of satellites for monitoring sea surface temperature (SST) by enabling SST data producers, users and scientists to collaborate within an agreed framework of best practice. GHRSST provides a framework for SST data sharing, best practices for data processing and a forum for scientific dialog, bringing SST to the user. The GHRSST format follows specifications for specific utilisation in the Earth Observation domain, providing semantics in the same domain to improve interoperability between SST products from different agencies.


Each product includes:

* measurement data files (NetCDF-4 format)

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>

 1. [Definining satellite names](#section1)
 1. [Setting query parameters](#section2)
 1. [Launching an OpenSearch request](#section3)

<hr>

In [7]:
# library imports 

import requests
import xml.dom.minidom
import warnings

warnings.filterwarnings('ignore')

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Defining satellite names
[Back to top](#TOC_TOP)

</div>

This first section allows to set the satellite names in order to define a related OpenSearch query

In [8]:
# setting satellite names
SST_MetOpA = "avhrr_sst_metop_a-osisaf-l2p-v1.0"
SST_MetOpB = "avhrr_sst_metop_b-osisaf-l2p-v1.0"
SST_MetOpC = "avhrr_sst_metop_c-osisaf-l2p-v1.0"

<div class="alert alert-info" role="alert">

## <a id='section2'></a>2. Setting query parameters
[Back to top](#TOC_TOP)

</div>

Setting Date, Time, Satellite and Lat°xLon° box.
Configure your selection

In [9]:
YYYY = "2022"
MM = "07"
DD = "09"
satellite = SST_MetOpB
lon_min = "-8"
lon_max = "3"
lat_min = "45"
lat_max = "51"

Setting connection information

In [10]:
session = requests.Session()
username = ''
password = ''

<div class="alert alert-info" role="alert">

## <a id='section3'></a>3. Launching an OpenSearch request
[Back to top](#TOC_TOP)

</div>

In [11]:
opensearch = requests.get('https://opensearch.ifremer.fr/granules.atom?'\
                          'datasetId=' + satellite + \
                          '&startPage=0&count=1000&'\
                          'timeStart='+YYYY+'-'+MM+'-'+DD+'T00:00:00Z&'\
                          'timeEnd='+YYYY+'-'+MM+'-'+DD+'T23:59:59Z&'\
                          'geoBox='+lon_min+','+lat_min+','+lon_max+','+lat_max, verify=False)
searchresult = open('searchresult.xml','w')
searchresult.write(opensearch.text)
searchresult.close()

In [12]:
# parse granules and download files
xml_resultfile = xml.dom.minidom.parse('searchresult.xml')
link_list = xml_resultfile.getElementsByTagName('link')
granule_number = link_list.length / 3
print("%d granules" % granule_number)
for link in link_list:
    if link.getAttribute('title') == "HTTPS" :
        url = link.getAttribute('href')
        print(url)
        r = requests.get(url, auth=(username, password), verify=False)
        # r.raw.decode_content = True
        filename = url.rsplit('/', 1)[1]
        print(filename)
        with open('data/' + filename, 'wb') as f:
           f.write(r.content)

7 granules
https://osi-saf.ifremer.fr/sst/l2p/global/avhrr_metop_b/2022/190/20220709190703-OSISAF-L2P_GHRSST-SSTsubskin-AVHRR_SST_METOP_B-sstmgr_metop01_20220709_190703-v02.0-fv01.0.nc
20220709190703-OSISAF-L2P_GHRSST-SSTsubskin-AVHRR_SST_METOP_B-sstmgr_metop01_20220709_190703-v02.0-fv01.0.nc
https://osi-saf.ifremer.fr/sst/l2p/global/avhrr_metop_b/2022/190/20220709110103-OSISAF-L2P_GHRSST-SSTsubskin-AVHRR_SST_METOP_B-sstmgr_metop01_20220709_110103-v02.0-fv01.0.nc
20220709110103-OSISAF-L2P_GHRSST-SSTsubskin-AVHRR_SST_METOP_B-sstmgr_metop01_20220709_110103-v02.0-fv01.0.nc
https://osi-saf.ifremer.fr/sst/l2p/global/avhrr_metop_b/2022/190/20220709091903-OSISAF-L2P_GHRSST-SSTsubskin-AVHRR_SST_METOP_B-sstmgr_metop01_20220709_091903-v02.0-fv01.0.nc
20220709091903-OSISAF-L2P_GHRSST-SSTsubskin-AVHRR_SST_METOP_B-sstmgr_metop01_20220709_091903-v02.0-fv01.0.nc
https://osi-saf.ifremer.fr/sst/l2p/global/avhrr_metop_b/2022/190/20220709092203-OSISAF-L2P_GHRSST-SSTsubskin-AVHRR_SST_METOP_B-sstmgr_metop0

<hr>
<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_?_link_to_prev_notebook.ipynb" target="_blank"><< PREV NOTEBOOK</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_?_link_to_next_notebook.ipynb" target="_blank">NEXT NOTEBOOK >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>